# Feelify Song Lyric Downloading
Author: Nathan Hunsberger

I'm just trying to use Genius API to download song lyrics for all the songs in MuSe

In [1]:
import pandas as pd
import re
import string
import requests
from tqdm import tqdm
tqdm.pandas()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
ERROR_FETCHING_LYRICS = "Error fetching lyrics"

def fix_name(name):
    index_of_feat = name.find("feat.")
    if index_of_feat > 0:
      name = name[:index_of_feat-1]
      name = name.strip()
    pattern = r'[{}]'.format(re.escape(string.punctuation))

    name = name.replace("&", "and")
    name = name.replace("-", " ")
    name = name.replace("/", " ")
    name = name.replace("ü", "u")
    name = name.replace("í", "i")
    name = name.replace("’", "")
    name = name.replace("ø", "o")
    name = name.replace("$", "")
    name = name.replace("ő", "o")
    name = re.sub(pattern, '', name)
    name = name.replace(' ','-')

    return name


# Thank u to whoever tf unlocked the Genius API over at weeb-api.vercel.app
def get_lyrics(artistname, songname):
    global ERROR_FETCHING_LYRICS
    artistname = str(artistname)
    songname = str(songname)
    artistname2 = fix_name(artistname)
    songname2 = fix_name(songname)
    url_genius = 'https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics'
    url = f"https://weeb-api.vercel.app/lyrics?url={url_genius}"
    #url = f"https://api.lyrics.ovh/v1/{artistname2}/{songname2}"
    # print(url_genius)
    try:
      response = requests.get(url)
      if response.status_code == 200:
          data = response.json()
          cleaned_data = re.sub(r'\[(.|\n)*?\]', '', data, flags=re.MULTILINE)
          return cleaned_data
      else:
          return ERROR_FETCHING_LYRICS
    except Exception as e:
      return ERROR_FETCHING_LYRICS

# Test to make sure API is working :)
print(get_lyrics("$UICIDEBOY$", "You’re Now Tuning in to 66.6 FM with DJ Rapture (The Hottest Hour of the Evening)"))



Sink back in that fucking spine, second-guessing crime

7th Ward where I snort a slug and cross the fucking line

West Bank/Northside 'till the coffin ride

Who am I? I am God; I am Basquiat



Trap-a-holics mixtapes

Man

, drop this shit for these fuck niggas



Grey*59 signed

Rough diamonds

, tryna shine
Christ and I, $ui-$uicide

You know, you know, you know I'm
Diving head first when crucified

Lucifer cried when I told him I'm choosing to die
Noose and a knife, but I ain't use the knife
To loosen the noose, keep it tight
Abusing the truth, that's a lie
Fuckboy with a ski mask think he gon' rob me blind?
Fuckboy better think fast when he cock the nine
Fuckboy better—
Fuckboy better, sink back in that fucking spine, second-guessin' crime

7th Ward where I snort a slug and cross the fucking line

Yet another line inside the cup

, 

another line you should look up

Another line that shook you up, I signed the line for Lucifer

Damn son, where'd you find this?

Real trap shit!





In [5]:
muse_df = pd.read_csv('drive/My Drive/CS 3892/muse_v3_lyrics_90000.csv')

In [11]:
muse_df.tail()

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics
89996,https://www.last.fm/music/gaby%2bhoffmann%2b%2...,Battle,Gaby Hoffmann & Jay Duplass,['transparent'],1,5.370000,3.450000,5.330000,NaN,NaN,NaN,Error fetching lyrics
89997,https://www.last.fm/music/omar%2brodriguez-lop...,Hands Tied to the Roots of a Hemorrhage,Omar Rodriguez-Lopez,['transparent'],4,5.797887,4.132254,5.570563,NaN,NaN,progressive rock,Error fetching lyrics
89998,https://www.last.fm/music/ace%2bwhite/_/crookf...,Crookfield Zoo,Ace White,['transparent'],1,5.370000,3.450000,5.330000,NaN,NaN,NaN,Error fetching lyrics
89999,https://www.last.fm/music/second%2bchyld/_/tra...,Transparent (Full Version),Second Chyld,['transparent'],1,5.370000,3.450000,5.330000,NaN,NaN,NaN,Error fetching lyrics
90000,https://www.last.fm/music/message%2bto%2bbears...,Unfold,Message To Bears,['translucent'],2,3.340000,1.405000,3.500000,247179b0-af1f-4c2b-b4fe-751070d1c0cc,14BYEOeRb3jzglJmKX8eM2,ambient,Error fetching lyrics


In [66]:
import concurrent.futures
import requests
import re
import pandas as pd
from tqdm import tqdm

# Function to apply get_lyrics to each row
def get_lyrics_for_row(row):
    return get_lyrics(row['artist'], row['track'])

# Get last half of lyrics
muse_df_45000 = muse_df[40000:]

# Prepare the list of rows for processing
rows = [row for _, row in muse_df_45000.iterrows()]
results = []

# Parallel execution with tqdm progress bar
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use tqdm to create a progress bar
    results = list(tqdm(executor.map(get_lyrics_for_row, rows), total=len(rows)))
    print('Finished pulling all lyrics')

# Assign the results back to the DataFrame
muse_df.loc[muse_df.index[45000:], 'lyrics'] = results

# Save the updated DataFrame
muse_df.to_csv('drive/My Drive/CS 3892/muse_v3_lyrics_90000.csv', index=False)

100%|██████████| 10001/10001 [33:33<00:00,  4.97it/s]


finished


In [67]:
muse_df['lyrics'].value_counts()["Error fetching lyrics"]

41718

Here i am messing around w pandas to make sure what we have is valid

In [75]:
muse_df.loc[muse_df.index[89410]]

lastfm_url                https://www.last.fm/music/scott%2bmatthews/_/c...
track                                                         City Headache
artist                                                       Scott Matthews
seeds                                                             ['quiet']
number_of_emotion_tags                                                    5
valence_tags                                                       6.024516
arousal_tags                                                       3.301613
dominance_tags                                                     6.112258
mbid                                   c7d7b15c-8cd6-4b3e-973f-28117078a53d
spotify_id                                           00OVRRNyPASjy9uN4pp2jQ
genre                                                                  folk
lyrics                    City headache subsides your dreary tone\nSend ...
Name: 89410, dtype: object

In [8]:
has_spotify_and_lyrics = muse_df.dropna(subset=['spotify_id', 'lyrics'], how='any')
has_spotify_and_lyrics = has_spotify_and_lyrics[has_spotify_and_lyrics['lyrics'] != ERROR_FETCHING_LYRICS]

In [9]:
has_spotify_and_lyrics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38975 entries, 0 to 89986
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lastfm_url              38975 non-null  object 
 1   track                   38975 non-null  object 
 2   artist                  38975 non-null  object 
 3   seeds                   38975 non-null  object 
 4   number_of_emotion_tags  38975 non-null  int64  
 5   valence_tags            38975 non-null  float64
 6   arousal_tags            38975 non-null  float64
 7   dominance_tags          38975 non-null  float64
 8   mbid                    34790 non-null  object 
 9   spotify_id              38975 non-null  object 
 10  genre                   37954 non-null  object 
 11  lyrics                  38975 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 3.9+ MB


In [10]:
len(has_spotify_and_lyrics[has_spotify_and_lyrics['artist'] == "Taylor Swift"])

35